# APE:  kookboek

Dit notebook is een speelveld voor Anonimisatie Pseudonimisatie & Encryptie (_APE_) technieken

De basis voor de verschillende technieken wordt in dit notebook gedemonstreerd a.h.v. standaard Python bibliotheken.

## Inhoud:
* [Bron bestand](#Bronbestand)
* [Verwijderen van een kolom](#Verwijderenvaneenkolom)
* [Pseudoniemensleutel](#Pseudoniemensleutel)
* [Pseudoniemensleutel tabel](#Pseudoniemensleuteltabel)
* [Veralgemening](#Veralgemening)
* [Fake names](#Fakenames)
* [Hashing](#Hashing)
* [Encryptie](#Encryptie)
    * [Genereer een sleutel](#Genereereensleutel)
    * [Encryptie van een string](#Encryptievaneenstring)
    * [Encryptie van een bestand](#Encryptievaneenbestand)
    * [Decryptie van een bestand](#Decryptievaneenbestand)


versie: 
* 2021-03: start
* 2021-05: uitbreiding met links

todo:
* duidelijke onderverdeling anonimiseren / pseudonimiseren? - volgen van het kookboek
* hashing paar voorbeelden voorzien

inspiratie: 
* https://korniichuk.medium.com/gdpr-guide-2-7c399b44ba3#adce
* https://developer.ibm.com/solutions/security/articles/s-gdpr3/
* https://www.anonos.com/enisa-chapter-1-introduction

## Bron bestand<a class="anchor" id="Bron_bestand"></a>

Een csv-bestand (ape_0.csv) dat persoonsgebonden gegevens bevat wordt gebruikt als startpunt. Het gaat hier om nepgegevens, deze werden gegenereerd op:  https://www.mockaroo.com/

Het bestand bevat volgende kolommen:
* id	
* first_name	
* last_name	
* email	
* gender	
* ip_address	
* city	
* country	
* age
* income
* score_1	
* score_2

De gegevens worden geïmporteerd als `pandas dataframe`

In [ ]:
import pandas as pd

# reading CSV input
df = pd.read_csv('ape_0.csv')
df

In [ ]:
type(df)

In [ ]:
type(df.email)

### Verwijderen van een kolom<a class="anchor" id="Verwijderenvaneenkolom"></a>

Als gegevens niet nodig zijn binnen een studie, verwijder deze dan. Hou vast aan dataminimalisatie. 

*drop* de onnodige kolom(men). In dit voorbeeld wordt de kolom `email` verwijderd.

In [ ]:
df.drop(columns=["email"], inplace=True)
df

#### Bewaar een dataframe

* dataframe dat enkel de nodige informatie bevat, wordt weggeschreven in een .csv bestand.  
* Verwijder het originele bestand als dit niet meer nodig is en werk verder met het bestand dat de minimale informatie bevat. 

In [ ]:
df.to_csv('ape_0_drop.csv')  

### Pseudoniemensleutel<a class="anchor" id="Pseudoniemensleutel"></a>

Een pseudoniemensleutel (token) is een pseudoniem dat wordt gebruikt in plaats van de oorspronkelijke gegevens.

In dit voorbeeld wordt de `uuid` bibliotheek gebruikt. Universal Unique Identifier, is een python bibliotheek die helpt bij het genereren van willekeurige unieke id's.

zie:
* https://pynative.com/python-uuid-module-to-generate-universally-unique-identifiers/
* https://docs.python.org/3/library/uuid.html


In onderstaand voorbeeld worden de originele gegevens uit de kolom `email`overschreven.


In [ ]:
import pandas as pd
import uuid

def email_tokenization(email):
    if email not in key:
        token = uuid.uuid4()
        while token in key.values():  # the token must be unique
            token = uuid.uuid4()
        key[email] = token
        return token
    else:
        return key[email]

# reading CSV input
df = pd.read_csv('ape_0.csv')

key = {}
df.email = df.email.map(email_tokenization)  # original email address is overwritten
df

In [ ]:
type(key)

In [ ]:
df.to_csv('ape_0_token.csv')  

### Pseudoniemensleutel tabel<a class="anchor" id="Pseudoniemensleuteltabel"></a>

De code kan aangepast worden om een pseudoniemensleutel tabel te maken.
Deze tabel zal het mogelijk maken om terug te keren naar de originele gegevens. Bewaar deze tabel veilig en apart van de gepseudonimiseerde gegevens.

In onderstaand voorbeeld worden de originele gegevens gebruikt.

In [ ]:
import pandas as pd
import uuid

# reading CSV input
df = pd.read_csv('ape_0.csv')

def email_tokenization(email):
    if email not in key:
        token = uuid.uuid4()
        while token in key.values():  # the token must be unique
            token = uuid.uuid4()
        key[email] = token
        return token
    else:
        return key[email]



key = {}
df['emailT'] = df.email.map(email_tokenization)  # a new column is created with the tokens

# create the lookup table and save it
df_lookup = df[['email', 'emailT']]
df_lookup.to_csv('lookup_0_email_token.csv') 

# remove the original email column
df.drop(columns=['email'], inplace=True)

df_lookup

In [ ]:
whos

### Veralgemening <a class="anchor" id="Veralgemening"></a>

Onderstaand voorbeeld is een eenvoudig voorbeeld over het veralgemenen van numerieke gegevens. 
De waarden van kolom `age` worden gecontroleerd en een string wordt teruggeplaatst als resultaat. Idem voor de numerieke gegevens van de kolom `income`.

In [ ]:
import pandas as pd

def age_generalization(age):
    if age < 18:
        return 'Age <= 18'
    else:
        return 'Age > 18'
    
def income_generalization(income):
    if income < 24000:
        return 'Below Average'
    else:
        return 'Above Average'

# reading CSV input
df = pd.read_csv('ape_0.csv')


df.age = df.age.map(age_generalization)
df.income = df.income.map(income_generalization)
df
df.to_csv('ape_0_generalization.csv')  

### Fake names <a class="anchor" id="Fakenames"></a>

De `faker`bibliotheek laat toe om snel nepgegevens aan te maken en deze kunnen dan gebruikt worden als pseudoniem. De originele gegevens kunnen overschreven worden, ofwel kan er een koppeltabel gecreëerd worden die dan veilig en apart van de gepseudonimiseerde gegevens moet bewaard worden.

In dit voorbeeld wordt de kolom `email` vervangen door nep emailadressen.

In [ ]:
import pandas as pd
from faker import Faker

def email_pseudonymization(email):
    if email not in key:
        pseudonym = fake.email()
        while (pseudonym in key.values()) or (pseudonym in key):
            pseudonym = fake.email()
        key[email] = pseudonym
        return pseudonym
    else:
        return key[email]

# reading CSV input
df = pd.read_csv('ape_0.csv')

key = {}
fake = Faker()
df.email = df.email.map(email_pseudonymization)
df
df.to_csv('ape_0_fakenames.csv')  

### Hashing <a class="anchor" id="Hashing"></a>

Onderstaand voorbeeld gebruikt de `hashlib`bibliotheek om gegevens te hashen.
* https://docs.python.org/3/library/hashlib.html
* https://pymotw.com/3/hashlib/

Binnen de `hashlib`bibliotheek zijn er 3 belangrijke funties:
* `update()`– De string die je wilt versleutelen moet gebruikt worden als argument in de functie.  
* `encode()`– de update() functie vereist een gecodeerde string, d.w.z. een opeenvolging van bits, daarom is de utf-8 codering of b nodig
* `hexdigest()`– Deze functie wordt gebruikt om de versleutelde string te genereren.



In [ ]:
import pandas as pd
import hashlib

def email_hashing(email):
    if email not in key:
        sha3 = hashlib.sha3_512()
        data = salt + email
        sha3.update(data.encode('utf-8'))
        hexdigest = sha3.hexdigest()
        key[email] = hexdigest
        return hexdigest
    else:
        return key[email]

# reading CSV input
df = pd.read_csv('ape_0.csv')

salt = 'medium'
key = {}
df.email = df.email.map(email_hashing)
df

## Encryptie <a class="anchor" id="Encryptie"></a>

bron: https://www.geeksforgeeks.org/encrypt-and-decrypt-files-using-python/

De `cryptography` bibliotheek zal worden gebruikt in onderstaande voorbeelden om gegevens te versleutelen. De `cryptography` bibliotheek gebruikt een symmetrisch algoritme om het bestand te versleutelen. Bij een symmetrisch algoritme wordt dezelfde sleutel gebruikt om het bestand te versleutelen en te ontsleutelen. De `fernet` module van het pakket heeft ingebouwde functies voor het genereren van de sleutel, encrypteren en decrypteren met respectievelijk de `encrypt()` en `decrypt()` methoden. De `fernet` module garandeert dat gegevens die met deze module zijn versleuteld niet verder kunnen worden gemanipuleerd of gelezen zonder de kennis van de sleutel. 



### Genereer een sleutel <a class="anchor" id="Genereereensleutel"></a>

Een *sleutel* wordt gebruikt om tekst te versleutelen, om een sterke sleutel te hebben kan deze worden gegenereerd door de software bibliotheek. 

In onderstaand voorbeeld wordt de sleutel weggeschreven in een bestand. Dit bestand moet zorgvuldig bewaard worden, want deze sleutel is nodig om de gegevens te decrypteren en de toegang tot deze sleutel moet heel erg beperkt zijn.

In [ ]:
# import required module 
from cryptography.fernet import Fernet

In [ ]:
# key generation 
key = Fernet.generate_key() 
  
# string the key in a file 
with open('filekey.key', 'wb') as filekey: 
   filekey.write(key)

### Encryptie van een string <a class="anchor" id="Encryptie_van_een_string"></a>

Voorbeeld van de encryptie van een string ('My secret message'). 

In [ ]:
message = "My secret message".encode() # bytes
# initialize the Fernet class
f = Fernet(key)
# encrypt the message
encrypted = f.encrypt(message)
# print how it looks
print(encrypted)

In [ ]:
decrypted_encrypted = f.decrypt(encrypted)
print(decrypted_encrypted)
original_message = decrypted_encrypted.decode() # bytes to string
print(original_message)

### Encryptie van een bestand <a class="anchor" id="Encryptievaneenbestand "></a>

In dit voorbeeld wordt een volledig bestand geëncrypteerd. Hier wordt het dus via een Python biblotheek functie gedaan, maar het kan evengoed met software zoals:

* 7zip
* AEScript

Volgende stappen zijn nodig in dit voorbeeld om een bestand met een sleutel te encrypteren: 

* Open het bestand dat de sleutel bevat.
* Initialiseer het Fernet object en sla het op in de fernet variabele.
* Importeer (lees) het originele bestand.
* Versleutel het bestand en sla het op in een object.
* Schrijf vervolgens de versleutelde gegevens in een bestand

In [ ]:
# opening the key 
with open('filekey.key', 'rb') as filekey: 
    key = filekey.read() 
  
# using the generated key 
fernet = Fernet(key) 
  
# opening the original file to encrypt 
with open('ape_0.csv', 'rb') as file: 
    original = file.read() 
      
# encrypting the file 
encrypted = fernet.encrypt(original) 
  
# opening the file in write mode and  
# writing the encrypted data 
with open('ape_0_encrypted.csv', 'wb') as encrypted_file: 
    encrypted_file.write(encrypted) 

### Decryptie van een bestand <a class="anchor" id="Decryptievaneenbestand "></a>


In dit voorbeeld wordt een geëncrypteerd bestand gedecrypteerd. 

Volgende stappen zijn nodig in dit voorbeeld om een bestand met een sleutel te encrypteren: 

* Open het bestand dat de sleutel bevat (moet dezelfde sleutel zijn als degen waarmee het bestand geëncrypteerd werd)
* Initialiseer het Fernet object en sla het op in de fernet variabele.
* Importeer (lees) het geëncrypteerde bestand.
* Decrypteer het bestand.
* Schrijf vervolgens gegevens in een bestand

In [ ]:
# opening the key 
with open('filekey.key', 'rb') as filekey: 
    key = filekey.read() 

# using the key 
fernet = Fernet(key) 
  
# opening the encrypted file 
with open('ape_0_encrypted.csv', 'rb') as enc_file: 
    encrypted = enc_file.read() 
  
# decrypting the file 
decrypted = fernet.decrypt(encrypted) 
  
# opening the file in write mode and 
# writing the decrypted data 
with open('ape_0_decrypt.csv', 'wb') as dec_file: 
    dec_file.write(decrypted) 